#Telecom Domain ReadOps Assignment
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql

create catalog if not exists telecom_catalog_assign;
create database if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;

**1.5 Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):**

**1. Volume Vs DBFS/filestore**

DBFS/FileStore is a legacy, workspace-level storage mainly used for temporary files, testing, or demos. It does not provide governance, fine-grained access control, or auditing, so it is not recommended for production use.

Databricks Volumes, on the other hand, are a Unity Catalog–governed storage layer used to store files securely in cloud storage. Volumes support fine-grained permissions, auditing, and external locations, making them production-ready and enterprise-grade.

**2. Why production teams prefer Volumes for regulated data?**

Production teams prefer Volumes because they provide better security, control, and tracking for sensitive or regulated data.

Only authorized people can access the data (user/group permissions)
Every access is tracked and audited -** Data is stored securely in cloud storage**
Access rules are centrally managed using Unity Catalog
Because of this, Volumes help organizations follow compliance rules (like GDPR, HIPAA, etc.) and avoid data misuse or leaks.

##2. Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

In [0]:
customer_csv = ''' 101,Arun,31,Chennai,PREPAID 102,Meera,45,Bangalore,POSTPAID 103,Irfan,29,Hyderabad,PREPAID 104,Raj,52,Mumbai,POSTPAID 105,,27,Delhi,PREPAID 106,Sneha,abc,Pune,PREPAID '''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count 101\t320\t1500\t20 102\t120\t4000\t5 103\t540\t600\t52 104\t45\t200\t2 105\t0\t0\t0 '''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp 5001|101|TWR01|-80|2025-01-10 10:21:54 5004|104|TWR05|-75|2025-01-10 11:01:12 '''

##3. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
'''path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol"
dbutils.fs.put(f"{path}/customer.csv", customer_csv, True)
dbutils.fs.put(f"{path}/usage.csv", usage_tsv, True)
dbutils.fs.put(f"{path}/tower_logs_region1.csv", tower_logs_region1, True)
dbutils.fs.put(f"{path}/tower_logs_region2.csv", tower_logs_region1, True)'''

dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv', customer_csv, True)
dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv', usage_tsv, True)
dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv',tower_logs_region1,True)
dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/tower_logs_region1.csv',tower_logs_region1,True)


In [0]:
# 3.2. Write a command to validate whether files were successfully copied

print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/'))
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/'))
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1')) 

##4. Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

##5. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

##6. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 